# Validaciones y exploración de los datos

In [1]:
# --- Celda 0: inicialización (ejecutar primero) ---
import os
import requests
import tempfile
from dotenv import load_dotenv

# Paquetes que Spark debe cargar (ajusta versiones si es necesario)
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages net.snowflake:spark-snowflake_2.12:3.1.2,'
    'net.snowflake:snowflake-jdbc:3.24.2 pyspark-shell'
)

# Ahora sí importamos pyspark y creamos la sesión
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

# Evita sesiones múltiples: si existe, detenla y crea una nueva limpia
if 'spark' in globals():
    try:
        spark.stop()
    except Exception:
        pass

conf = SparkConf().setAppName("NYC_TLC_ingest").setMaster("local[*]")
# opcional: conf.set("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:3.1.2,net.snowflake:snowflake-jdbc:3.24.2")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Comprueba
print("Spark inicializado:", spark.version)



Spark inicializado: 3.5.0


In [2]:
from dotenv import load_dotenv
import os

load_dotenv("/home/jovyan/work/.env", override=True)

account = os.getenv("SNOWFLAKE_ACCOUNT")
sf_url = os.getenv("SNOWFLAKE_URL") or account
if sf_url and not sf_url.endswith("snowflakecomputing.com"):
    sf_url = f"{sf_url}.snowflakecomputing.com"

if not all([account, os.getenv("SNOWFLAKE_USER"), os.getenv("SNOWFLAKE_PASSWORD")]):
    raise RuntimeError("Faltan variables de Snowflake en /home/jovyan/work/.env")

print("sfURL:", sf_url)




sfURL: LKVTWCT-PPC14557.snowflakecomputing.com


In [7]:
# Chunk PySpark: validación de calidad de datos para OBT_TRIPS
import os
from dotenv import load_dotenv

load_dotenv()

# --- SF options ---
sfOptions = {
    "sfURL": os.getenv("SNOWFLAKE_URL") or os.getenv("SNOWFLAKE_ACCOUNT"),
    "sfUser": os.getenv("SNOWFLAKE_USER"),
    "sfPassword": os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase": os.getenv("SNOWFLAKE_DATABASE"),
    "sfSchema": os.getenv("SNOWFLAKE_SCHEMA", "BRONZE"),
    "sfWarehouse": os.getenv("SNOWFLAKE_WH"),
    "sfRole": os.getenv("SNOWFLAKE_ROLE"),
}

database = sfOptions["sfDatabase"]
target_schema = "ANALYTICS"

# Consultas de validación
validation_queries = {
    "1_nulos_por_columna": f"""
    SELECT 
        COUNT(*) AS total_registros,
        SUM(CASE WHEN INGEST_RUN_ID IS NULL THEN 1 ELSE 0 END) AS ingest_run_id_nulos,
        SUM(CASE WHEN PICKUP_DATETIME IS NULL THEN 1 ELSE 0 END) AS pickup_datetime_nulos,
        SUM(CASE WHEN DROPOFF_DATETIME IS NULL THEN 1 ELSE 0 END) AS dropoff_datetime_nulos,
        SUM(CASE WHEN TRIP_DISTANCE IS NULL THEN 1 ELSE 0 END) AS trip_distance_nulos,
        SUM(CASE WHEN FARE_AMOUNT IS NULL THEN 1 ELSE 0 END) AS fare_amount_nulos,
        SUM(CASE WHEN TOTAL_AMOUNT IS NULL THEN 1 ELSE 0 END) AS total_amount_nulos,
        SUM(CASE WHEN PASSENGER_COUNT IS NULL THEN 1 ELSE 0 END) AS passenger_count_nulos,
        SUM(CASE WHEN PULOCATIONID IS NULL THEN 1 ELSE 0 END) AS pulocationid_nulos,
        SUM(CASE WHEN DOLOCATIONID IS NULL THEN 1 ELSE 0 END) AS dolocationid_nulos
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    """,
    
    "2_rangos_numericos": f"""
    SELECT 
        -- Trip Distance (millas)
        MIN(TRIP_DISTANCE) AS min_distance_miles,
        MAX(TRIP_DISTANCE) AS max_distance_miles,
        AVG(TRIP_DISTANCE) AS avg_distance_miles,
        COUNT(CASE WHEN TRIP_DISTANCE <= 0 THEN 1 END) AS distance_cero_o_negativa,
        COUNT(CASE WHEN TRIP_DISTANCE > 100 THEN 1 END) AS distance_mayor_100_millas,
        
        -- Fare Amount
        MIN(FARE_AMOUNT) AS min_fare_amount,
        MAX(FARE_AMOUNT) AS max_fare_amount,
        AVG(FARE_AMOUNT) AS avg_fare_amount,
        COUNT(CASE WHEN FARE_AMOUNT < 0 THEN 1 END) AS fare_negativo,
        COUNT(CASE WHEN FARE_AMOUNT > 1000 THEN 1 END) AS fare_mayor_1000,
        
        -- Passenger Count
        MIN(PASSENGER_COUNT) AS min_passengers,
        MAX(PASSENGER_COUNT) AS max_passengers,
        AVG(PASSENGER_COUNT) AS avg_passengers,
        COUNT(CASE WHEN PASSENGER_COUNT <= 0 THEN 1 END) AS passengers_cero_o_negativo,
        COUNT(CASE WHEN PASSENGER_COUNT > 10 THEN 1 END) AS passengers_mayor_10,
        
        -- Trip Duration (minutos)
        MIN(TRIP_DURATION_MIN) AS min_duration_min,
        MAX(TRIP_DURATION_MIN) AS max_duration_min,
        AVG(TRIP_DURATION_MIN) AS avg_duration_min,
        COUNT(CASE WHEN TRIP_DURATION_MIN <= 0 THEN 1 END) AS duration_cero_o_negativo,
        COUNT(CASE WHEN TRIP_DURATION_MIN > 180 THEN 1 END) AS duration_mayor_3_horas,
        
        -- Average Speed (MPH)
        MIN(AVG_SPEED_MPH) AS min_speed_mph,
        MAX(AVG_SPEED_MPH) AS max_speed_mph,
        AVG(AVG_SPEED_MPH) AS avg_speed_mph,
        COUNT(CASE WHEN AVG_SPEED_MPH > 100 THEN 1 END) AS speed_mayor_100_mph
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    """,
    
    "3_coherencia_fechas": f"""
    SELECT 
        COUNT(*) AS total_registros,
        -- Fechas inconsistentes
        SUM(CASE WHEN DROPOFF_DATETIME < PICKUP_DATETIME THEN 1 ELSE 0 END) AS dropoff_antes_pickup,
        SUM(CASE WHEN PICKUP_DATETIME > CURRENT_DATE() THEN 1 ELSE 0 END) AS pickup_en_futuro,
        SUM(CASE WHEN DROPOFF_DATETIME > CURRENT_DATE() THEN 1 ELSE 0 END) AS dropoff_en_futuro,
        
        -- Rangos de fechas
        MIN(PICKUP_DATETIME) AS fecha_pickup_min,
        MAX(PICKUP_DATETIME) AS fecha_pickup_max,
        MIN(DROPOFF_DATETIME) AS fecha_dropoff_min,
        MAX(DROPOFF_DATETIME) AS fecha_dropoff_max,
        
        SUM(CASE WHEN YEAR(PICKUP_DATETIME) != YEAR THEN 1 ELSE 0 END) AS anio_inconsistente,
        SUM(CASE WHEN MONTH(PICKUP_DATETIME) != MONTH THEN 1 ELSE 0 END) AS mes_inconsistente
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    """,
    
    "4_conteos_mes_servicio": f"""
    SELECT 
        SERVICE_TYPE,
        YEAR,
        MONTH,
        COUNT(*) AS total_viajes,
        SUM(PASSENGER_COUNT) AS total_pasajeros,
        AVG(TRIP_DISTANCE) AS avg_distance_miles,
        AVG(FARE_AMOUNT) AS avg_fare_amount,
        AVG(TRIP_DURATION_MIN) AS avg_duration_min,
        SUM(TOTAL_AMOUNT) AS ingreso_total
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    GROUP BY SERVICE_TYPE, YEAR, MONTH
    ORDER BY YEAR, MONTH, SERVICE_TYPE
    """,
    
    "5_validacion_zonas": f"""
    SELECT 
        -- Pickup zones
        COUNT(DISTINCT PULOCATIONID) AS unique_pulocationid,
        SUM(CASE WHEN PICKUP_ZONE IS NULL THEN 1 ELSE 0 END) AS pickup_zone_nulos,
        SUM(CASE WHEN PICKUP_BOROUGH IS NULL THEN 1 ELSE 0 END) AS pickup_borough_nulos,
        
        -- Dropoff zones
        COUNT(DISTINCT DOLOCATIONID) AS unique_dolocationid,
        SUM(CASE WHEN DROPOFF_ZONE IS NULL THEN 1 ELSE 0 END) AS dropoff_zone_nulos,
        SUM(CASE WHEN DROPOFF_BOROUGH IS NULL THEN 1 ELSE 0 END) AS dropoff_borough_nulos,
        
        -- Zonas más comunes
        (SELECT PICKUP_ZONE FROM "{database}"."{target_schema}"."OBT_TRIPS" GROUP BY PICKUP_ZONE ORDER BY COUNT(*) DESC LIMIT 1) AS pickup_zone_mas_comun,
        (SELECT DROPOFF_ZONE FROM "{database}"."{target_schema}"."OBT_TRIPS" GROUP BY DROPOFF_ZONE ORDER BY COUNT(*) DESC LIMIT 1) AS dropoff_zone_mas_comun
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    """,
    
    "6_validacion_pagos": f"""
    SELECT 
        PAYMENT_TYPE,
        PAYMENT_TYPE_DESC,
        COUNT(*) AS total_viajes,
        AVG(FARE_AMOUNT) AS avg_fare,
        AVG(TIP_AMOUNT) AS avg_tip,
        AVG(TIP_PCT) AS avg_tip_pct,
        SUM(TOTAL_AMOUNT) AS ingreso_total
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    GROUP BY PAYMENT_TYPE, PAYMENT_TYPE_DESC
    ORDER BY total_viajes DESC
    """,
    
    "7_resumen_servicios": f"""
    SELECT 
        SERVICE_TYPE,
        COUNT(*) AS total_viajes,
        COUNT(DISTINCT VENDORID) AS vendors_unicos,
        AVG(TRIP_DISTANCE) AS avg_distance_miles,
        AVG(TRIP_DURATION_MIN) AS avg_duration_min,
        AVG(PASSENGER_COUNT) AS avg_passengers,
        SUM(TOTAL_AMOUNT) AS ingreso_total
    FROM "{database}"."{target_schema}"."OBT_TRIPS"
    GROUP BY SERVICE_TYPE
    ORDER BY total_viajes DESC
    """
}

print("Iniciando validación de calidad de datos para OBT_TRIPS...")

try:
    import snowflake.connector
    import pandas as pd
    
    print("✅ snowflake-connector-python y pandas están instalados")
    
    # Extraer información de conexión
    account = sfOptions['sfURL']
    if '.snowflakecomputing.com' in account:
        account = account.split('.snowflakecomputing.com')[0]
    
    # Conectar a Snowflake
    print("Conectando a Snowflake...")
    conn = snowflake.connector.connect(
        user=sfOptions['sfUser'],
        password=sfOptions['sfPassword'],
        account=account,
        warehouse=sfOptions['sfWarehouse'],
        database=sfOptions['sfDatabase'],
        schema=target_schema,
        role=sfOptions['sfRole']
    )
    
    print("✅ Conexión establecida")
    
    # Crear cursor
    cursor = conn.cursor()
    
    # Ejecutar todas las validaciones
    results = {}
    
    for query_name, query in validation_queries.items():
        try:
            print(f"\n🔍 Ejecutando validación: {query_name}")
            cursor.execute(query)
            
            # Obtener resultados
            if query_name in ["4_conteos_mes_servicio", "6_validacion_pagos", "7_resumen_servicios"]:
                # Para queries con múltiples filas, usar pandas
                df = cursor.fetch_pandas_all()
                results[query_name] = df
                print(f"✅ {query_name} - {len(df)} filas obtenidas")
                if len(df) > 0:
                    print(df.head().to_string())
            else:
                # Para queries de resumen, mostrar directamente
                result = cursor.fetchone()
                results[query_name] = result
                columns = [desc[0] for desc in cursor.description]
                print(f"✅ {query_name} - Resultados:")
                for col, val in zip(columns, result):
                    print(f"   {col}: {val}")
        except Exception as e:
            print(f"❌ Error en validación {query_name}: {str(e)}")
            results[query_name] = None
    
    # Cerrar conexión
    cursor.close()
    conn.close()
    
    print("\n" + "="*80)
    print("🎉 VALIDACIÓN COMPLETADA - RESUMEN EJECUTIVO")
    print("="*80)
    
    # Resumen ejecutivo con manejo seguro de errores
    try:
        # Total de registros
        if results.get("1_nulos_por_columna"):
            total_registros = results["1_nulos_por_columna"][0]
            print(f"\n📊 TOTAL DE REGISTROS: {total_registros:,}")
        else:
            total_registros = 0
            print(f"\n📊 TOTAL DE REGISTROS: No disponible")
        
        # Principales hallazgos
        nulos_pickup = results.get("1_nulos_por_columna", [0, 0, 0, 0])[2] if results.get("1_nulos_por_columna") else 0
        nulos_dropoff = results.get("1_nulos_por_columna", [0, 0, 0, 0])[3] if results.get("1_nulos_por_columna") else 0
        dropoff_antes_pickup = results.get("3_coherencia_fechas", [0, 0])[1] if results.get("3_coherencia_fechas") else 0
        
        print(f"\n🚨 PRINCIPALES HALLAZGOS:")
        if total_registros > 0:
            print(f"   • Registros con pickup datetime nulo: {nulos_pickup} ({nulos_pickup/total_registros*100:.2f}%)")
            print(f"   • Registros con dropoff datetime nulo: {nulos_dropoff} ({nulos_dropoff/total_registros*100:.2f}%)")
            print(f"   • Viajes con dropoff antes del pickup: {dropoff_antes_pickup} ({dropoff_antes_pickup/total_registros*100:.4f}%)")
        else:
            print(f"   • No se pudieron calcular los principales hallazgos")
            
        # Métricas adicionales de rangos numéricos
        if results.get("2_rangos_numericos"):
            rangos = results["2_rangos_numericos"]
            print(f"\n📈 MÉTRICAS DE DISTANCIAS:")
            print(f"   • Distancia mínima: {rangos[0]:.2f} millas")
            print(f"   • Distancia máxima: {rangos[1]:.2f} millas")
            print(f"   • Distancia promedio: {rangos[2]:.2f} millas")
            print(f"   • Viajes con distancia ≤ 0: {rangos[3]:,}")
            print(f"   • Viajes con distancia > 100 millas: {rangos[4]:,}")
    
    except Exception as summary_error:
        print(f"❌ Error generando resumen ejecutivo: {str(summary_error)}")
    
    print(f"\n✅ Proceso de validación finalizado")
    
except ImportError as e:
    print(f"❌ Error de importación: {e}")
    print("Instala las dependencias con: pip install snowflake-connector-python pandas")
except Exception as e:
    print(f"❌ Error durante la validación: {str(e)}")
    import traceback
    print(f"Detalle del error: {traceback.format_exc()}")

Iniciando validación de calidad de datos para OBT_TRIPS...
✅ snowflake-connector-python y pandas están instalados
Conectando a Snowflake...
✅ Conexión establecida

🔍 Ejecutando validación: 1_nulos_por_columna
✅ 1_nulos_por_columna - Resultados:
   TOTAL_REGISTROS: 847739473
   INGEST_RUN_ID_NULOS: 0
   PICKUP_DATETIME_NULOS: 0
   DROPOFF_DATETIME_NULOS: 0
   TRIP_DISTANCE_NULOS: 0
   FARE_AMOUNT_NULOS: 0
   TOTAL_AMOUNT_NULOS: 0
   PASSENGER_COUNT_NULOS: 17833894
   PULOCATIONID_NULOS: 0
   DOLOCATIONID_NULOS: 0

🔍 Ejecutando validación: 2_rangos_numericos
✅ 2_rangos_numericos - Resultados:
   MIN_DISTANCE_MILES: -40840124.4
   MAX_DISTANCE_MILES: 59016609.3
   AVG_DISTANCE_MILES: 5.652642092566569
   DISTANCE_CERO_O_NEGATIVA: 7936576
   DISTANCE_MAYOR_100_MILLAS: 18035
   MIN_FARE_AMOUNT: -133391414.0
   MAX_FARE_AMOUNT: 998310.03
   AVG_FARE_AMOUNT: 13.70909867597967
   FARE_NEGATIVO: 3714647
   FARE_MAYOR_1000: 687
   MIN_PASSENGERS: 0
   MAX_PASSENGERS: 192
   AVG_PASSENGERS: 1.554